## Providing Input-Discriminative Protection for Local Differential Privacy

## 一、背景简介

在本地化差分隐私机制中，服务器被认为是不可信的第三方，每个用户使用满足LDP的扰动机制独立地将自身的原始数据进行扰动。服务器收集用户扰动后的数据进行分析。本地化差分隐私目前已经广泛应用于实践中，例如Google的Chrome以及苹果的Safari都使用了LDP。


<img src="images/LocalModel.png" height="300" width="400" >



目前，基于一致隐私预算的LDP有：RAPPOR，Optimized Unary Encoding（OUE），SVIM（for set-valued data），以及PrivKV（for key-value data）等等，这些机制对于所有的输入都使用相同的隐私预算$\epsilon$。然而，在一些实际场景中我们期望可以对不同输入提供不同的隐私预算，例如在医学领域，一般期望对于HIV、癌症等疾病的隐私保护程度高于感冒等普通疾病，如果使用相同的隐私预算，则会使得无需过多保护的信息使用了较强的隐私保护，这无疑会降低数据的可用性。现有的机制无法处理该问题，例如，personalized local differential privacy (PLDP)只提供了用户级别的不同隐私预算，而geo-indistinguishability只能处理基于不同位置时的情境。

因此，本文提出一种输入区分的本地化差分隐私机制（ID-LDP），该机制可以对不同的输入$x \in D$，提供不同的隐私预算$\epsilon _{x}$。


本文的主要贡献有如下几点：

（1）提出了一种名为ID-LDP以及基于此的实例MinID-LDP，它可以提供比LDP更加细粒度的隐私保护；

（2）设计了一种IDUE机制，它满足MinID-LDP，可以提供对于输入频率的无偏估计。为了优化IDUE的MSE（Mean Square Error），本文形式化了目标函数以此解决最小化方差的无偏估计，并得到了三个实际的优化模型；

（3）将IDUE与Padding-and-Sampling（PS）结合，设计了IDUE-PS机制，它满足MinID-LDP，并且与IDUE具有相同的计算复杂度；

（4）通过实际的数据集验证了IDUE和IDUE-PS，结果显示本文提出的机制对于类别数据的频率估计有更好的表现。


## 二、ID-LDP

#### 1.定义（Input-Discriminative LDP（ID-LDP））
对于一个给定的隐私预算集合$\boldsymbol {\varepsilon}  = \{\epsilon _{x}\}, x\in D$，D为输入数据的值域。一个随机化的机制M满足$\boldsymbol {\varepsilon} $ -ID-LDP，当且仅当对于任意的输入$x，x'\in D$，以及任意的输出$y \in Range(M)$,有:

$$\frac{Pr(M(x) = y)}{Pr(M(x') = y)} \leq e^{r(\epsilon _{x}, \epsilon _{x'})}$$


其中$r(\cdot,\cdot)$是关于差分隐私预算的二元函数。

#### 2.定义（MinID-LDP）
一个随机化的机制M满足$\boldsymbol {\varepsilon} $-MinID-LDP当且仅当它满足$r(\epsilon _{x}, \epsilon _{x'})=min(\epsilon _{x}, \epsilon _{x'})$的$\boldsymbol {\varepsilon} $-ID-LDP，其中$\boldsymbol {\varepsilon} $为所有输入的隐私预算的集合。

#### 3.定理（MinID-LDP的序列组合定理）

如果对于$i=1,2,3,...,k$，随机化机制$M_{i} : D \rightarrow R_{i}$，满足$\boldsymbol {\varepsilon} _{i}$-MinID-LDP，其中$\boldsymbol {\varepsilon} _{i} = \{\epsilon_{x}^{(i)}\}$，则$M=(M_{1},M_{2},...,M_{k}) : D \rightarrow R_{1}\times R_{2}\times ... \times R_{k}$，满足$(\sum ^{k} _{i=1} {\boldsymbol {\varepsilon} _{i}} )$-MinID-LDP，其中$(\sum _{i=1}^{k} \boldsymbol {\varepsilon} _{i}) = \{ \sum _{i=1}^{k} {\epsilon_{x}^{(i)}} \}$。


证明：设$x, x' \in D$，$y=(y_{1}, y_{2},...,y_{k}) \in R_{1}\times R_{2}\times ... \times R_{k}$,则有


$$\frac{Pr(M(x)=y)}{Pr(M(x')=y)} = \prod _{i=1}^{k} \frac{Pr(M_{i}(x)=y_{i})}{Pr(M_{i}(x')=y_{i})} \leq \prod _{i=1}^{k} e ^{\min \{ \epsilon _{x} ^{(i)}, \epsilon _{x'} ^{(i)} \}} \leq \prod _{i=1}^{k} e ^{ \epsilon _{x} ^{(i)}} = e^{\sum _{i=1}^{k} \epsilon _{x} ^{(i)}}$$

同理可得：
$$\frac{Pr(M(x)=y)}{Pr(M(x')=y)} \leq e^{\sum _{i=1}^{k} \epsilon _{x'} ^{(i)}}$$

因此：

$$\frac{Pr(M(x)=y)}{Pr(M(x')=y)} \leq e^{\min \{ \sum _{i=1}^{k} \epsilon _{x} ^{(i)}, \sum _{i=1}^{k} \epsilon _{x'} ^{(i)} \} } $$

#### 4. 与其它的LDP的比较

<img src="images/ALL_LDP.png" height="400" width="600" >

#### 5. Min-LDP 与LDP的关系

对于所有的$x \in D$，若$\epsilon _{x} = \epsilon$，则$\boldsymbol {\varepsilon} $-MinID-LDP 为$\epsilon$-LDP。因此，$\epsilon$-LDP是$\boldsymbol {\varepsilon} $-MinID-LDP的一个特例。

引理 ：如果一个机制满足$\epsilon$-LDP，则它也满足$\boldsymbol {\varepsilon} $-MinID-LDP，其中对于所有的$\boldsymbol {\varepsilon} $有$\min\{\boldsymbol {\varepsilon}\} = \epsilon$。另一方面，如果一个机制满足$\boldsymbol {\varepsilon} $-MinID-LDP，则它满足$\epsilon$-LDP，其中$\epsilon = min\{\max\{ \boldsymbol {\varepsilon} \}, 2\min\{ \boldsymbol {\varepsilon}\}\}$


证明：

由LDP与MinID-LDP定义可以直接推出如下结果：

$$\min\{ \boldsymbol {\varepsilon} \}-LDP \Rightarrow \boldsymbol {\varepsilon}-MinID-LDP \qquad (1)$$


$$\boldsymbol{\varepsilon}-MinID-LDP \Rightarrow \max\{ \boldsymbol {\varepsilon} \}-LDP \qquad (2)$$


因此只需要证明如下结论：
$$\boldsymbol{\varepsilon}-MinID-LDP \Rightarrow 2\min\{ \boldsymbol {\varepsilon} \}-LDP \qquad (3)$$


设 $\epsilon_{x^{*}} = \min \{ \boldsymbol {\varepsilon} \}$，即$x^{*}$的隐私预算最小，则有：

$$ \frac{Pr(M(x)=y)}{Pr(M(x')=y)} = \frac{Pr(M(x)=y)}{Pr(M(x^{*})=y)} \frac{Pr(M(x^{*})=y)}{Pr(M(x')=y)} \leq e^{\min\{\epsilon_{x}, \epsilon_{x^{*}}\}} e^{\min\{\epsilon_{x'}, \epsilon_{x^{*}}\}}  = e ^{2\min\{ \boldsymbol {\varepsilon} \}}$$

#### 6.先验-后验隐私泄露分析

根据论文Local Information Privacy中的定义，

<img src="images/LIP.png" height="300" width="400" style="float:left" >


可以通过如下方式分析先验-后验隐私泄露分析。

$$ \frac{Pr(x)}{Pr(x|y)} = \frac{Pr(y)}{Pr(y|x)}  = \frac{\sum_{x' \in D} Pr(x')Pr(y|x')}{Pr(y|x)} = \sum_{x' \in D} Pr(x') \frac{Pr(y|x')}{Pr(y|x)} $$

下图表示各种LDP的先验-后验概率比，它们有类似的上下界，但是当输入不同时，MinID-LDP有不同的上下界

<img src="images/ALL_LDP_bpp.png" height="300" width="400" >

#### 7.其它ID-LDP

除了MinID-LDP外，也可以定义AvgID-LDP，即$r(\epsilon _{x}, \epsilon _{x'}) = (\epsilon _{x}+ \epsilon _{x'})/2$。AvgID-LDP满足序列组合定理，并且可以应用于如下IDUE和IDUE-PS。

## 三、IDUE

#### 0.问题描述

每个用户拥有隐私数据$x \in \{1,2,...,m\}$，服务器端期望通过满足ID-LDP机制实现对用户数据$x$进行频率统计。

#### 1. Input-Discriminative Unary Encoding (IDUE)

对于每一个单项输入$x=i$，我们使用如下$m$长向量的编码方式：

$$\boldsymbol{x}=\boldsymbol{v}_{i} = [0,...,0,1,0,...,0]$$


其中$\boldsymbol{v}_{i}$向量中，第i位为1，其余位为0。接下来使用如下方式进行扰动：

$$Pr(y[k]=1|x[k]=1) = a_{k} \qquad  Pr(y[k]=0|x[k]=1) = 1-a_{k}$$

$$Pr(y[k]=1|x[k]=0) = b_{k} \qquad Pr(y[k]=0|x[k]=0) = 1-b_{k}$$


为了保证数据的有用性，令$a_{k} > b_{k}$

由此可以得到如下等式：


$$\frac{Pr(\boldsymbol{y}|x=i)}{Pr(\boldsymbol{y}|x'=j)} =\frac{ \prod _{k=1}^{m} Pr(\boldsymbol{y}[k]|x=i)}{\prod _{k=1}^{m} Pr(\boldsymbol{y}[k]|x'=j)} = \frac{Pr(\boldsymbol{y}[i]|\boldsymbol{v}_{i})Pr(\boldsymbol{y}[j]|\boldsymbol{v}_{i})}{Pr(\boldsymbol{y}[i]|\boldsymbol{v}_{j})Pr(\boldsymbol{y}[j]|\boldsymbol{v}_{j})} = \frac{Pr(\boldsymbol{y}[i]|\boldsymbol{v}_{i}[i]=1)Pr(\boldsymbol{y}[j]|\boldsymbol{v}_{i}[j]=0)}{Pr(\boldsymbol{y}[i]|\boldsymbol{v}_{j}[i]=0)Pr(\boldsymbol{y}[j]|\boldsymbol{v}_{j}[j]=1)}$$


又由于$a_{k} > b_{k}，1-b_{k}> 1-a_{k}$，则：

$$\frac{Pr(\boldsymbol{y}[i]|\boldsymbol{v}_{i})Pr(\boldsymbol{y}[j]|\boldsymbol{v}_{i})}{Pr(\boldsymbol{y}[i]|\boldsymbol{v}_{j})Pr(\boldsymbol{y}[j]|\boldsymbol{v}_{j})}= \frac{(\frac{a_{i}}{b_{i}}) ^{y[i]} (\frac{1-a_{i}}{1-b_{i}}) ^{1-y[i]}}{(\frac{a_{j}}{b_{j}}) ^{y[j]} (\frac{1-a_{j}}{1-b_{j}}) ^{1-y[j]}} \leq \frac{a_{i}(1-b_{j})}{b_{i}(1-a_{j})}$$


要使得IDUE满足ID-LDP，则有：

$$\frac{a_{i}(1-b_{j})}{b_{i}(1-a_{j})} \leq e^{r(\epsilon _{i}, \epsilon _{j})}$$

IDUE用于输入为频繁项的频繁统计的算法步骤：

<img src="images/IDUE.png" height="300" width="400" style="float:left" >

#### 2. 频率估计 

若$c_{i} = \sum _{u \in U}\boldsymbol{y}_{u}[i]$，则我们使用如下公式进行频率估计：

$$\hat{c}_{i} = \frac{c_{i} - nb_{i}}{a_{i}-b_{i}}$$


无偏性：

$E[c_{i}] = c^{*}_{i}a_{i} + \sum _{k \ne i}c^{*}_{k}b_{i} = c^{*}_{i}a_{i} + (n-c^{*}_{i})b_{i} $

所以 $E[\hat{c}_{i}] = \frac{E[c_{i}] - nb_{i}}{a_{i}-b_{i}} = c^{*}_{i}$，即$\hat{c}_{i}$是$c_{i}^{*}$的无偏估计

$MSE$：

$$MSE_{\hat{c}_{i}} = E[\hat{c}_{i} - c^{*}_{i}]^{2} = Var[\hat{c}_{i}]= \frac{Var[c_{i}]}{(a_{i}-b_{i})^{2}}=\frac{c^{*}_{i}a_{i}(1-a_{i}) + (n-c^{*}_{i})b_{i}(1-b_{i})}{(a_{i}-b_{i})^{2}} = \frac{nb_{i}(1-b_{i})}{(a_{i}-b_{i})^{2}} + \frac{c^{*}_{i}(1-a_{i}-b_{i})}{a_{i}-b_{i}}$$

#### 3. 最小化MSE

设输入项的集合为$I= \{I_{1}, I_{2},...,I_{t}\}$,其中$I_{i}$表示隐私预算为$\epsilon _{i}（i=1,2,...,t）$的项的集合,$m_{i}$表示$I_{i}$中项的个数。则对于$I_{i}$中的所有项，将使用相同的参数$(a_{i},b_{i})$，$I_{i}$的MSE可以通过如下等式计算：


$$MSE_{I_{i}}= \frac{nm_{i}b_{i}(1-b_{i})}{(a_{i}-b_{i})^{2}} + \frac{(1-a_{i}-b_{i})}{a_{i}-b_{i}} \sum _{k \in I_{i}}c_{k}^{*}$$


##### opt0: 最差情况

由于$\sum _{i=1}^{t} \sum _{k \in I_{i}} c_{k}^{*} \leq n$，则有如下不等式：


$$\sum _{i=1}^{t} MSE_{I_{i}} \leq \sum _{i=1}^{t} \frac{nm_{i}b_{i}(1-b_{i})}{(a_{i}-b_{i})^{2}} + \max \{\frac{1-a_{i} - b_{i}}{a_{i}-b_{i}}\} \cdot n$$

上式表示的是最差情况，我们期望的是最差情况越小越好，因此最终得到的目标函数为：

$$ \min_{a,b} f \triangleq \sum _{i=1}^{t} \frac{nm_{i}b_{i}(1-b_{i})}{(a_{i}-b_{i})^{2}} + \max \{\frac{1-a_{i} - b_{i}}{a_{i}-b_{i}}\} \cdot n $$

$$s.t. \frac{a_{i}(1-b_{j})}{b_{i}(1-a_{j})} \leq e^{r(\epsilon _{i}, \epsilon _{j})} (\forall i,j = 1,2,...,t) $$

$$0<b_{i}<a_{i}<1 (\forall i,j = 1,2,...,t) $$


由于上式的可行解包含RAPPOR和OUE，因此上式的最优解情况要优于RAPPOR和OUE。上式并不是一个凸函数，因此无法直接求最优解，但是可以通过进一步约束条件，使其成为一个凸优化问题。

##### opt1: RAPPOR结构

通过限制$a_{i} + b_{i} = 1 (\forall i)$，则可以用如下公式表示$a_{i}$和$b_{i}$

$$a_{i} = \frac{e ^{\tau _{i}}}{1+e ^{\tau _{i}}},\quad b_{i} = \frac{1}{1+e ^{\tau _{i}}} \qquad (i=1,2,...,t)$$


其中$\tau _{i} > 0$，则总的$MSE$为：

$$\sum _{i=1}^{t} MSE_{I_{i}} \leq \sum _{i=1}^{t} \frac{nm_{i}b_{i}(1-b_{i})}{(a_{i}-b_{i})^{2}} = n\sum _{i=1}^{t} \frac{m_{i}e^{\tau_{i}}}{(e^{\tau_{i}}-1)^{2}}$$

约束条件为：

$$\frac{a_{i}(1-b_{j})}{b_{i}(1-a_{j})} = e^{\tau_{i}+\tau_{j}} \leq e^{r(\epsilon _{i}, \epsilon _{j})} \Leftrightarrow \tau_{i}+\tau_{j} \leq r(\epsilon _{i}, \epsilon _{j})$$ 


因此最终的目标函数为：

$$ \min_{\tau_{1},\tau_{2},...,\tau_{t}} f \triangleq \sum _{i=1}^{t} \frac{m_{i}e^{\tau_{i}}}{(e^{\tau_{i}}-1)^{2}}$$

$$s.t.\quad \tau_{i}+\tau_{j} \leq r(\epsilon _{i}, \epsilon _{j}) \quad (\forall i,j = 1,2,...,t) $$


##### opt2: OUE结构

令$a_{i} = 1/2(\forall i)$,则$\frac{1-a_{i} - b_{i}}{a_{i}-b_{i}} = 1(\forall i)$

$$\frac{a_{i}(1-b_{j})}{b_{i}(1-a_{j})} = \frac{1-b_{j}}{b_{i}} \leq  e^{r(\epsilon _{i}, \epsilon _{j})} \Leftrightarrow  e^{r(\epsilon _{i}, \epsilon _{j})} \cdot b_{i} + b_{j} \leq 1$$


$$\sum _{i=1}^{t} MSE_{I_{i}}= \sum _{i=1}^{t} \frac{nm_{i}b_{i}(1-b_{i})}{(a_{i}-b_{i})^{2}} + \frac{(1-a_{i}-b_{i})}{a_{i}-b_{i}} \sum _{k \in I_{i}}c_{k}^{*} = \sum _{i=1}^{t}\frac{nm_{i}b_{i}(1-b_{i})}{(0.5-b_{i})^{2}}+n$$


因此，此时的目标函数为：

$$ \min_{0<b_{i}<0.5} f(b) \triangleq  \sum _{i=1}^{t}\frac{m_{i}b_{i}(1-b_{i})}{(0.5-b_{i})^{2}}$$
$$s.t. \quad  e^{r(\epsilon _{i}, \epsilon _{j})} \cdot b_{i} + b_{j} \leq 1$$


#### 4.ID-LDP与LDP比较

<img src="images/ALL_LDP_var.png" >

## 四、IDUE-PS

#### 0.问题描述

每个用户拥有一个隐私数据数据集$x \subseteq \{1,2,...,m\}$，服务器端期望通过满足ID-LDP机制实现对用户数据$x$进行频率统计。

#### 1.PS协议

PS协议(Padding-and-Sampling protocol)是基于LDP的频繁项集挖掘算法，由于输入为频繁项集，每一个项都需要分配隐私预算。因此PS协议采取了数据填充与截断采样的方式，将频繁项集的项个数固定为特定长度，从而避免隐私预算需要过多分配的问题。

<img src="images/PS.png" height="300" width="400" style="float:left" >

#### 2.IDUE-PS协议

算法步骤：

<img src="images/IDUE_PS.png" height="300" width="400" style="float:left" >


<img src="images/IDUE_PS_s.png" height="300" width="500" style="float:left" >

由于上述过程中对样本以概率为$\frac{1}{l}$进行采样，因此最终的频率估计为：$\hat {c}_{i} = l \cdot \frac{c_{i} - nb_{i}}{a_{i} - b_{i}}$


#### 3.__引理__：

对于输入项集$x,x' \in D$，IDUE-PS的任意输出有如下表达式的界：

$$\frac{Pr(y|x)}{Pr(y|x')} \leq \frac{\eta _{x} \sum_{i \in x} \frac{\alpha _{i}}{|x|} + (1-\eta _{x}) \sum_{i=m+1}^{m+l}\frac{\alpha _{i}}{l}}{\eta _{x'} \sum_{j \in x'} \frac{\beta _{j}}{|x|} + (1-\eta _{x'}) \sum_{i=m+1}^{m+l}\frac{\beta _{j}}{l}}$$





由于填充的为无效项，无需设置不同的隐私保护参数，因此：$a_{i}=a^{*}, b_{i} = b^{*} (i=m+1,...,m+l)$,上述界可以重写为：

$$\frac{Pr(y|x)}{Pr(y|x')} \leq \frac{\eta _{x} \sum_{i \in x} \frac{\alpha _{i}}{|x|} + (1-\eta _{x}) \alpha ^{*}}{\eta _{x'} \sum_{j \in x'} \frac{\beta _{j}}{|x|} + (1-\eta _{x'}) \beta ^{*}}$$


对于每一个不同的输入项集，则可以设置隐私隐私预算$\epsilon _{x}$为如下值：

$$\epsilon _{x} = ln(\eta _{x} \sum_{i \in x} \frac{e^{\epsilon _{i}}}{|x|} + (1-\eta _{x}) e^{ \epsilon ^{*}})$$



__证明：__

<img src="images/IDUE_PS_bound.png" height="300" width="500" style="float:left" >

#### 4.引理

如果IDUE满足MinID-LDP机制，即：

$$\frac{\alpha _{i}}{\beta _{j}} = \frac{a_{i}(1-b_{j})}{b_{i}(1-a_{j})} \leq e^{\min \{\epsilon _{i}, \epsilon _{j}\}} \quad (\forall i,j \in I)$$

则IDUE-PS也满足MinID-LDP机制，即：

$$\frac{Pr(y|x)}{Pr(y|x')} \leq e^{\min \{\epsilon _{x}, \epsilon _{x'}\}} \quad (\forall x,x' \in D, \forall y)$$


__证明：__

<img src="images/IDUE_to_IDUE_PS.png" height="300" width="500"  style="float:left">



调和平均数 ≤ 几何平均数 ≤ 算术平均数 ≤ 平方平均数

## 五、实验结果

<img src="images/ID_LDP_RE.png" >